### Running DESeq2

As I went looking for single-cell differential expression packages, I came across [this paper](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2599-6) and unabashedly skipped to the conclusion which reported that "methods developed specifically for scRNAseq data do not show significantly better performance compared to the methods designed for bulk RNAseq data; and methods that consider behavior of each individual gene (not all genes) in calling DE genes outperform the other tools."

Luckily, I have already used DESeq2 (which is designed for bulk-seq data). It also _is_ one of the methods that considers the behavior of each individual gene (according to the same paper). For those reasons, I'm applying it here. 



In [1]:
library("DESeq2")

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min



Attaching package: ‘S4Vectors’


The

Define a path prefix:

In [2]:
prefix <- "/data/codec/production.run/mrna/"

Define the directory with the counts and then a results directory.

In [3]:
countsdir <- paste(prefix,"de.csvs/pseudobulks/",sep="")
resdir <- paste(prefix,"deseq2.res/pseudobulks/",sep="")

List the files in `countsdir`.

In [4]:
sampleFiles <- list.files(countsdir)

In [5]:
sampleFiles

[1] "A.col.csv" "A.csv"     "B.col.csv" "B.csv"     "G.col.csv" "G.csv"    
 [7] "P.col.csv" "P.csv"     "R.col.csv" "R.csv"

File names are returned in alphabetical order, so the `col.csv` always comes before the `cts.csv`.

In [7]:
for (cond in c("A","B","G","P","R")) {
    
    cond_files <- grep(cond, sampleFiles, value = TRUE)
    
    counts <- as.matrix(read.csv(paste(countsdir, 
                                    cond_files[2], # using the second position, which is the cts.csv
                                    sep=""
                                   ),row.names=1, check.names = FALSE))
    
    coldata <- read.csv(paste(countsdir, 
                              cond_files[1], # using the first position, which is the col.csv
                              sep=""
                             ), row.names=1)
    
    # reclassify as factor, right now reading at an integer
    coldata$FID <- as.factor(coldata$FID)
    
# #     # first, run the time course model outlined in the rnaseqgene vignette
# #     ddsTC <- DESeqDataSetFromMatrix(countData = cts,
# #                     colData = coldata,
# #                                 design = ~ cond + TIME + cond:TIME
# #                                )
# #     ddsTC <- DESeq(ddsTC, test="LRT", reduced = ~ cond + TIME)
# #     res <- results(ddsTC)
# #     write.csv(as.data.frame(res), file=paste(resdir,cond,".TC.csv",sep=""))
    
    # then for each time point, do just regular differential expression between condulation and control

    dds <- DESeqDataSetFromMatrix(countData = counts,
                          colData = coldata,
                          design = ~ COND
                         )
    dds <- DESeq(dds, parallel = TRUE)
    res <- results(dds, contrast = c("COND",cond,"C"))
    write.csv(as.data.frame(res), file=paste(resdir, cond, ".csv",sep=""))
    
}

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates: 30 workers

mean-dispersion relationship

final dispersion estimates, fitting model and testing: 30 workers

-- replacing outliers and refitting for 31 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates: 30 workers

mean-dispersion relationship

final dispersion estimates, fitting model and testing: 30 workers

-- replacing outliers and refitting for 44 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates: 30 workers

mean-dispersion relationsh